In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import zipfile
for dirname, _, filenames in os.walk('/kaggle/input/quora-insincere-questions-classification/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import time
import os
import pandas as pd
import numpy as np
import random
import copy

import re
from gensim.models import KeyedVectors
import gc

from keras.preprocessing.text import Tokenizer

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

t0 = time.time()

data_path = '/kaggle/input/'
seed = 2077

# Any results you write to the current directory are saved as output.


/kaggle/input/quora-insincere-questions-classification/train.csv
/kaggle/input/quora-insincere-questions-classification/embeddings.zip
/kaggle/input/quora-insincere-questions-classification/test.csv
/kaggle/input/quora-insincere-questions-classification/sample_submission.csv
/kaggle/input/fairseq/full_translation (1).txt
/kaggle/input/beam-5/translation_3.txt
/kaggle/input/beam-5/translation_4.txt
/kaggle/input/beam-5/translation_2.txt
/kaggle/input/beam-5/translation_0.txt
/kaggle/input/beam-5/translation_1.txt
/kaggle/input/quora-insincere-questions-classification/train.csv
/kaggle/input/quora-insincere-questions-classification/embeddings.zip
/kaggle/input/quora-insincere-questions-classification/test.csv
/kaggle/input/quora-insincere-questions-classification/sample_submission.csv


Using TensorFlow backend.


In [2]:
train_df = pd.read_csv(data_path + 'quora-insincere-questions-classification/' + 'train.csv')

train_df, val_df = train_test_split(train_df.copy(), train_size=0.7, random_state=seed)

In [3]:
train_df.head()

,qid,question_text,target
487207,5f6988e5cb1096691086,When will men and boys learn the truth and go ...,1
1113645,da36c0a619e1cf3fcdd4,Why does not Keralites (India) does not like t...,0
558619,6d73fed2c64a4883e27b,Will you still incur a cash advance fee if you...,0
807057,9e22ba5c0a04d9a44236,What are the real advantages of using Quora?,0
1163578,e4006d45becca4c20ab7,How much coffee is safe to drink?,0


In [4]:
oneline = train_df[~train_df['question_text'].str.contains('\n')]
multiline = train_df[train_df['question_text'].str.contains('\n')]

In [5]:
for line in multiline['question_text'].values:
    print(line)
    print("_____________________________________")

What are the origins of the phrases "Light of my 
life" and "Fire of my loins"?
 
_____________________________________
http://youtu.be/5oqFm5XCQww
 Burmese army throws muslim children into fire infront of their mothers, Aung su kyi is silent...so is world. hypocrite much?
_____________________________________
Can Adrián Lamo [ http://www.opham.main.quora.com/profile/Adrián-Lamo ] hack like Hackerman from Kung Fury? https://www.youtube.com/watch?v=KEkrWRHCDQU

_____________________________________
In "Star Trek 2013" why did they :

*Spoilers*
*Spoilers*
*Spoilers*
*Spoilers*

1)Make warping look quite a bit like an hyperspace jump
2)what in the world were those bright particles as soon as they jumped.
3)Why in the world did they make it possible for two entities to react in warp space in separate jumps.
4)Why did Spock get emotions for this movie.
5)What was the point of hiding the "Enterprise" underwater.
6)When they were intercepted by the dark ship, how come they reached Earth when

In [6]:
np.savetxt(r'./questions.txt', oneline['question_text'].values, fmt='%s')

In [7]:
oneline.head()

,qid,question_text,target
487207,5f6988e5cb1096691086,When will men and boys learn the truth and go ...,1
1113645,da36c0a619e1cf3fcdd4,Why does not Keralites (India) does not like t...,0
558619,6d73fed2c64a4883e27b,Will you still incur a cash advance fee if you...,0
807057,9e22ba5c0a04d9a44236,What are the real advantages of using Quora?,0
1163578,e4006d45becca4c20ab7,How much coffee is safe to drink?,0


In [8]:
train_df.head()

,qid,question_text,target
487207,5f6988e5cb1096691086,When will men and boys learn the truth and go ...,1
1113645,da36c0a619e1cf3fcdd4,Why does not Keralites (India) does not like t...,0
558619,6d73fed2c64a4883e27b,Will you still incur a cash advance fee if you...,0
807057,9e22ba5c0a04d9a44236,What are the real advantages of using Quora?,0
1163578,e4006d45becca4c20ab7,How much coffee is safe to drink?,0


In [9]:
lines0 = pd.read_csv('/kaggle/input/beam-5/translation_0.txt', sep="\n", header=None)
lines1 = pd.read_csv('/kaggle/input/beam-5/translation_1.txt', sep="\n", header=None)
lines2 = pd.read_csv('/kaggle/input/beam-5/translation_2.txt', sep="\n", header=None)
lines3 = pd.read_csv('/kaggle/input/beam-5/translation_3.txt', sep="\n", header=None)
lines4 = pd.read_csv('/kaggle/input/beam-5/translation_4.txt', sep="\n", header=None)

In [10]:
! wc -l /kaggle/input/fairseq/full_translation.txt

wc: /kaggle/input/fairseq/full_translation.txt: No such file or directory


In [11]:
!head -n 15742 /kaggle/input/fairseq/full_translation.txt | tail -1

head: cannot open '/kaggle/input/fairseq/full_translation.txt' for reading: No such file or directory


In [12]:
lines = pd.read_csv('/kaggle/input/fairseq/full_translation (1).txt', sep="\n", header=None)

In [13]:
lines.head()

,0
0,When will men and boys learn the truth and go ...
1,Why does Keralites (India) not speak Hindi?
2,If you pay a $0 credit card and the company se...
3,What are the real benefits of using Quora?
4,How much coffee is safe to drink?


In [14]:
print(len(lines.values))
print(len(oneline.values))

914280
914280


In [15]:
#for fairseq
augmented = pd.DataFrame({'question_text' : list(lines.values), 
                          'target' : list(oneline['target'].values)})

In [16]:
augmented.to_csv('augmented.csv')